![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-getting-started.png)

In [9]:
import os
import shutil
import requests
import tempfile
import azureml.core
from azureml.core import Workspace, Experiment, Datastore, Environment
# from azureml.widgets import RunDetails
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep
from azureml.core.runconfig import DockerConfiguration
from azureml.core import Experiment

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

ws = Workspace.from_config()
env = Environment.get(workspace=ws, name="automl-nasa-test2")

# Enable Docker
docker_config = DockerConfiguration(use_docker=True)

print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')


experiment_name = 'train-on-nasaexperiment'
experiment = Experiment(workspace = ws, name = experiment_name)

SDK version: 1.38.0
mlprojecta
parta-projecta
koreacentral
e6ec79e7-c7e5-4312-85d0-75e8285c09dd


In [10]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

aml_compute_target1 = "cpu-cluster-automl"
try:
    aml_compute1 = AmlCompute(ws, aml_compute_target1)
    print("found existing compute target.")
except ComputeTargetException:
    print("creating new compute target")
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                                min_nodes = 1, 
                                                                max_nodes = 4)    
    aml_compute1 = ComputeTarget.create(ws, aml_compute_target1, provisioning_config)
    aml_compute1.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
print("Azure Machine Learning Compute attached")

aml_compute_target2 = "cpu-cluster-custom"
try:
    aml_compute2 = AmlCompute(ws, aml_compute_target2)
    print("found existing compute target.")
except ComputeTargetException:
    print("creating new compute target")
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                                min_nodes = 1, 
                                                                max_nodes = 4)    
    aml_compute2 = ComputeTarget.create(ws, aml_compute_target2, provisioning_config)
    aml_compute2.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
print("Azure Machine Learning Compute attached")


In [ ]:
# Use a RunConfiguration to specify some additional requirements for this step.
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

# create a new runconfig object
run_config = RunConfiguration()

# enable Docker 
run_config.environment.docker.enabled = True

# set Docker base image to the default CPU-based image
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE

# use conda_dependencies.yml to create a conda environment in the Docker image for execution
run_config.environment.python.user_managed_dependencies = False

# specify CondaDependencies obj
run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn', 'pandas', 'numpy', 'azure'])

In [ ]:
from azureml.core import ScriptRunConfig

# Uses default values for PythonScriptStep construct.

source_directory = './scripts'
print('Source directory for the step is {}.'.format(os.path.realpath(source_directory)))

os.makedirs(source_directory, exist_ok=True)
shutil.copy('../nasa_forecast/helper.py', source_directory)

# step1 = PythonScriptStep(name="Auto ML",
#                          script_name="auto-ml-forecasting-nasa.py", 
#                          compute_target=aml_compute1, 
#                          source_directory=source_directory,
#                          runconfig=run_config,
#                          allow_reuse=False)
src1 = ScriptRunConfig(source_directory=source_directory, 
                      script='auto-ml-forecasting-nasa.py', 
                      compute_target=aml_compute1, 
                      environment=env,
                      docker_runtime_config=docker_config)
 
run1 = experiment.submit(config=src1)

# All steps use the same Azure Machine Learning compute target as well
# step2 = PythonScriptStep(name="Custom ML",
#                          script_name="custom-ml-forecasting-nasa.py", 
#                          compute_target=aml_compute2, 
#                          source_directory=source_directory,
#                          runconfig=run_config,
#                          allow_reuse=False)
src2 = ScriptRunConfig(source_directory=source_directory, 
                      script='custom-ml-forecasting-nasa.py', 
                      compute_target=aml_compute2, 
                      environment=env,
                      docker_runtime_config=docker_config)

run2 = experiment.submit(config=src2) 

In [6]:
# list of steps to run
steps = [step1, step2]
print("Step lists created")

pipeline1 = Pipeline(workspace=ws, steps=steps)
print ("Pipeline is built")

NameError: name 'step1' is not defined

In [ ]:
pipeline1.validate()
print("Pipeline validation complete")

In [ ]:
pipeline_run1 = Experiment(ws, 'TimeSeriesMetric').submit(pipeline1, regenerate_outputs=False)
print("Pipeline is submitted for execution")

In [ ]:
RunDetails(pipeline_run1).show()